In [1]:
import re
import itertools
import pandas as pd
import numpy as np

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2

D:\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [4]:
def get_lda_vector(text, num_topics):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(num_topics):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [5]:
def get_user_embedding(user_articles_list, method):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = method(user_vector, 0)
    return user_vector

In [6]:
def fill_users_embedding(users, num_topics, method):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, method), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(num_topics)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(num_topics)]]
    return user_embeddings

In [7]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Новости

In [8]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [9]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [10]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

In [11]:
with open('stopwords.txt', encoding='utf-8') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

778

In [12]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-3-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 22.8 s


In [13]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 42s


А теперь обучим нашу модель

In [14]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [15]:
num_topics = 25

In [16]:
%%time
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=num_topics, id2word=common_dictionary)#, passes=10)

Wall time: 24.9 s


In [17]:
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [18]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.14122646),
 (9, 0.071772784),
 (10, 0.071214356),
 (14, 0.08790743),
 (16, 0.60680544)]

In [19]:
x=lda.show_topics(num_topics=num_topics, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: палата употребление регистрация проба найтись мэрия болгария
topic_1: тыс рубль млн исследование доллар пациент месяц
topic_2: фестиваль праздник программа рейтинг россиянин место гость
topic_3: гражданин составить млн млрд выяснить налог бюджет
topic_4: военный исследование научный операция армия территория российский
topic_5: конкурс супруг производить оплата греция остаток миллиардер
topic_6: пенсия двигатель пилот катастрофа достигать вуз знаменитый
topic_7: россия новый сша российский рынок эксперт развитие
topic_8: рак температура дождь градус рт диск дневный
topic_9: земля погибнуть население проект взрыв поверхность запуск
topic_10: украина украинский белоруссия киев белорусский донбасс минск
topic_11: журнал американский квартира участок метод предполагаться бомба
topic_12: испытание писать восток берег долг лодка энергия
topic_13: россия помощь москва агентство житель гражданин центр
topic_14: исследование планета снижение жизнь выяснить запустить общество
topic_15: 

С помощью функцию get_lda_vector, которая возвращает векторное представление новости, сформируем матрицу, в которой показано как каждая тема представлена в документе.<br>
Получим вектора наших новостей:

In [20]:
topic_matrix = pd.DataFrame([get_lda_vector(text, num_topics) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(num_topics)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(num_topics)]]
topic_matrix.head(3)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.076108,0.0,0.0,0.01284,0.0,0.061124,0.0,...,0.841739,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4896,0.0,0.102049,0.281669,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.000000,0.000000,0.070012,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4897,0.0,0.000000,0.141255,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.000000,0.606861,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(num_topics)]].values))

In [22]:
doc_dict[293672]

array([0.        , 0.19497253, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.16296634,
       0.        , 0.        , 0.03817326, 0.38183588, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02978495,
       0.        , 0.1711465 , 0.        , 0.        , 0.        ])

### Следующий шаг - векторные представления пользователей

In [23]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче.<br>
Будем использовать различные аггрегационные функции, указанные в списке method_list:

In [24]:
method_list = [np.mean, np.median, np.max]
preds_list = []

target = pd.read_csv("users_churn.csv")
logreg = LogisticRegression()

In [25]:
for method in method_list:
    user_embeddings = fill_users_embedding(users, num_topics, method=method)
    X = pd.merge(user_embeddings, target, 'left')
    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(num_topics)]], 
                                                    X['churn'], random_state=0)
    #обучим
    logreg.fit(X_train, y_train)
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    preds_list.append(preds)

In [26]:
preds_list

[array([0.12217263, 0.17929797, 0.39039115, ..., 0.0011084 , 0.03513577,
        0.00419845]),
 array([0.13962441, 0.05489526, 0.4156178 , ..., 0.00207116, 0.0449298 ,
        0.00153168]),
 array([2.48639876e-01, 9.33116383e-02, 8.22352740e-01, ...,
        3.04490542e-04, 3.16837883e-03, 9.61307456e-05])]

### Рассчитаем Precision, Recall, F_score

In [27]:
metrics_matrix = pd.DataFrame(columns=['thresholds', 'fscore', 'precision', 'recall', 'roc_auc'])

In [28]:
for ind, method in enumerate(method_list):
    precision, recall, thresholds = precision_recall_curve(y_test, preds_list[ind])
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, preds_list[ind])
    metrics_matrix.loc[str(method).split()[1]] = [thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score(y_test, preds_list[ind])]

In [29]:
metrics_matrix

,thresholds,fscore,precision,recall,roc_auc
mean,0.223133,0.596667,0.504225,0.730612,0.920947
median,0.302412,0.679487,0.713004,0.648980,0.943097
amax,0.354365,0.761905,0.773109,0.751020,0.973543


В целом мы видим, что получившиеся векторные представления содержат какой-то сигнал и позволяют решать нашу прикладную задачу. 

Результат по медиану и максимуму лучше, потому что больше характеризуют предпочтения пользователя по темам. 
Для примера возьмем 5 статей и представим, что 4 из них о спорте с небольшим весом, а пятый об экономике с высоким весом. 
Тогда и медиана и максимум более информативны, потому что определят либо наиболее часто читаемые темы, либо узконаправленные. 